In [1]:
import sys
sys.path.append("../src")
import pandas as pd
import divop
#import notebooks.divop as divop
import sys
sys.path.append("../src")
import imp
import warnings
warnings.filterwarnings("ignore")

In [2]:
def testing_synthetic_data(data,path_positives,path_negatives, n_iterations,n_positives,n_negatives,frac_positives=None,frac_negatives=None):
    #load the previous created synthetic training data
    test_data_positives=pd.read_csv(path_positives, index_col=0) #positives
    test_data_negatives= pd.read_csv(path_negatives, index_col=0) #negatives
    positives=test_data_positives[test_data_positives["Type"]=='g'].sample(n=n_positives,frac=frac_positives) 
    negatives=test_data_negatives[test_data_negatives["Type"]=='b'].sample(n=n_negatives,frac=frac_negatives)
    #define the dataframe containing the validation metrics (1 row for each synthetic dataset)
    metrics = pd.DataFrame(columns=['f1','precision','recall','auc'])
    for i in range(n_iterations):
        print(i)
        #creating the sequences
        df_synthetic_test = divop.collect_synthetic_data(data, iterations=1, n=4)#to create 1 dataset of synthetic data
        imp.reload(divop) #to empty the dataframes in the library
        #classifying the paths in the test set and calculating the validation metrics
        print('classifying synthetic data')
        classified_data, metrics_list = divop.classify_higher_order_paths(df_synthetic_test,n_negatives=negatives, n_positives=positives, synthetic=True, var_order=False)
        metrics.loc[i] = metrics_list #adding metrics results to the metrics df
        classified_data['cycle'] = i # we create a column containing, for each row, in which cycle it has been generated, to eventually evaluate validation metric later
        if i==0:
            classified_data_previous = classified_data
        elif i==(n_iterations-1):
            classified_data_all = pd.concat([classified_data_previous, classified_data],ignore_index=True)
            print('FINAL VALIDATION METRICS EVALUATION')
            print('f1 score: '+ str(metrics['f1'].mean()) + '+/-' + str(metrics['f1'].std()))
            print('precision score: '+ str(metrics['precision'].mean()) + '+/-' + str(metrics['precision'].std()))
            print('recall score: '+ str(metrics['recall'].mean()) + '+/-' + str(metrics['recall'].std()))
            print('auc score: '+ str(metrics['auc'].mean()) + '+/-' + str(metrics['auc'].std()))
        else:
            classified_data_all = pd.concat([classified_data_previous, classified_data],ignore_index=True)
            classified_data_previous = classified_data_all
        
    return classified_data_all,metrics

## Flight Data

In [3]:
data_flight = []
path_data = "../data/input/coupons_2018_01-5percent.ngram"
with open(path_data) as f:
    for line in f:
        line = line.split(",")
        chain = [str(x) for x in line[:-1]]
        data_flight.append(chain)

In [ ]:
path_positives = "../data/input/flight_positives_training_data.csv"
path_negatives = "../data/input/flight_negatives_training_data.csv.xz"
n_iterations=3
n_positives = None
n_negatives = None
#Here we test flight data training the algorithm through synthetic flight data
flight_testing_df, metrics = testing_synthetic_data(data_flight, path_positives, path_negatives, n_iterations,n_positives,n_negatives,frac_positives=1,frac_negatives=1)

### Testing flight data, after training the algorithm through orbis synthetic data

In [ ]:
path_positives = "../data/input/orbis_positives_training_data.csv.csv"
path_negatives = "../data/input/orbis_negatives_training_data.csv.xz"
n_iterations=3
n_positives = None
n_negatives = None
#NOTE!: here we are testing flight data training the algorithm through synthetic Orbis data. Even though we still obtain good results, it is not the tested and best way to go
#The correct way is to create training flight data and train the algorithm through it, however, you can test your data in this way if you want a sight of the performance thorugh this method
flight_testing_df, metrics = testing_synthetic_data(data_flight, path_positives, path_negatives, n_iterations,n_positives,n_negatives,frac_positives=1,frac_negatives=1)